In [1]:
import pandas as pd
pd.options.display.max_columns = None

In [2]:
orders = pd.read_csv("../data/2019-01-31 s24/order.csv", low_memory=False)
orders = orders.rename(index=str, columns={"id": "order_id"})
orders = orders.rename(columns=lambda name: name if name.endswith("id") else "order_" + name )
orders = orders[orders["order_deleted"] == 0] # drop deleted orders
orders = orders[[
    "order_id", "order_amount", "order_volume","order_deliver_at_start",
    "order_deliver_at_end", "order_fulfillment_type",
    "store_id", "courier_id", "picker_id", "customer_id"]]
orders.tail(5)

/home/gionata/github/analitico/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (9,17,21,22,23,25,26,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,order_id,order_amount,order_volume,order_deliver_at_start,order_deliver_at_end,order_fulfillment_type,store_id,courier_id,picker_id,customer_id
479948,980930,95.58,127.17,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963286,None,None,963322
479949,980932,178.63,29.85,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963288,None,None,963324
479950,980934,22.43,104.97,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963290,None,None,963326
479951,980936,83.99,180.87,2019-01-27 20:00:00,2019-01-27 21:00:00,1,963292,None,None,963328
479952,980938,90.39,86.90,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963294,None,None,963330


In [3]:
details = pd.read_csv("../data/2019-01-31 s24/order_detail.csv")
details = details.tail(50000)

/home/gionata/github/analitico/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,10,14,15,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# count number of items in each order
dt1 = details.order_id.value_counts().reset_index(name='items_total')
dt1.rename(index=str, columns={"index": "order_id"}, inplace=True)
dt1.set_index("order_id", inplace=True)
dt1.head(10)

# count number of items with variable weight
details_by_order = details.groupby(['order_id'])
dt2 = details_by_order['variable_weight'].sum().reset_index(name='items_with_variable_weight')
dt2.set_index("order_id", inplace=True)
dt2.tail(10)

orders = pd.merge(orders, dt1, on="order_id")
orders = pd.merge(orders, dt2, on="order_id")
orders.tail(10)

,order_id,order_amount,order_volume,order_deliver_at_start,order_deliver_at_end,order_fulfillment_type,store_id,courier_id,picker_id,customer_id,items_total,items_with_variable_weight
1930,980920,43.43,13.60,2019-01-27 15:00:00,2019-01-27 16:00:00,3,963276,64456,12,963312,7,0
1931,980922,34.91,31.57,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963278,None,None,963314,7,1
1932,980924,110.65,154.24,2019-01-27 10:00:00,2019-01-27 11:00:00,1,963280,None,None,963316,27,8
1933,980926,31.72,16.59,2019-01-26 17:00:00,2019-01-26 18:00:00,1,963282,None,None,963318,8,0
1934,980928,34.31,55.69,2019-01-26 18:00:00,2019-01-26 19:00:00,1,963284,None,None,963320,18,2
1935,980930,95.58,127.17,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963286,None,None,963322,37,8
1936,980932,178.63,29.85,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963288,None,None,963324,9,0
1937,980934,22.43,104.97,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963290,None,None,963326,9,0
1938,980936,83.99,180.87,2019-01-27 20:00:00,2019-01-27 21:00:00,1,963292,None,None,963328,44,2
1939,980938,90.39,86.90,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963294,None,None,963330,12,3


In [5]:
order_ts = pd.read_csv("../data/2019-01-31 s24/order_timestamps.csv")
order_ts = order_ts.rename(columns=lambda name: name if name == 'order_id' else "order_" + name )
order_ts = order_ts.set_index("order_id")
order_ts = order_ts.drop(["order_client_synced_at", "order_synced_at"], axis=1)
order_ts.tail(2000)

,order_opened_at,order_first_detail_picked_at,order_last_detail_picked_at,order_first_try_pay_at,order_delivered_at
order_id,,,,,
976934,2019-01-26 10:17:53,2019-01-26 10:17:53,2019-01-26 12:08:20,2019-01-26 12:13:10,2019-01-26 12:30:25
976936,2019-01-25 08:58:56,2019-01-25 09:03:57,2019-01-25 09:34:08,2019-01-25 09:42:51,2019-01-25 10:05:47
976938,2019-01-25 10:32:15,2019-01-25 10:34:21,2019-01-25 10:40:40,2019-01-25 10:46:21,2019-01-25 11:02:27
976940,2019-01-25 11:45:43,2019-01-25 11:49:46,2019-01-25 12:05:44,2019-01-25 12:14:00,2019-01-25 12:32:41
976942,2019-01-25 11:22:28,2019-01-25 11:26:35,2019-01-25 11:56:34,2019-01-25 12:13:16,2019-01-25 12:52:57
976944,2019-01-25 09:14:23,2019-01-25 09:16:39,2019-01-25 09:45:57,2019-01-25 10:03:24,2019-01-25 10:29:31
976946,2019-01-25 08:21:53,2019-01-25 08:25:06,2019-01-25 08:28:27,2019-01-25 09:31:21,2019-01-25 09:42:53
976948,2019-01-25 12:53:51,2019-01-25 13:46:42,2019-01-25 14:12:44,2019-01-25 14:24:57,2019-01-25 15:26:27
976950,2019-01-25 10:03:41,2019-01-25 10:10:23,2019-01-25 10:33:57,2019-01-25 10:47:19,2019-01-25 11:12:14


In [6]:
orders = pd.merge(orders, order_ts, on="order_id")
orders.tail(10)

,order_id,order_amount,order_volume,order_deliver_at_start,order_deliver_at_end,order_fulfillment_type,store_id,courier_id,picker_id,customer_id,items_total,items_with_variable_weight,order_opened_at,order_first_detail_picked_at,order_last_detail_picked_at,order_first_try_pay_at,order_delivered_at
1927,980914,34.99,115.40,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963270,None,None,963306,21,1,None,None,None,None,None
1928,980916,45.66,180.51,2019-01-27 09:00:00,2019-01-27 10:00:00,1,963272,None,None,963308,8,0,None,None,None,None,None
1929,980918,31.72,33.57,2019-01-27 12:00:00,2019-01-27 13:00:00,1,963274,None,None,963310,11,1,None,None,None,None,None
1930,980920,43.43,13.60,2019-01-27 15:00:00,2019-01-27 16:00:00,3,963276,64456,12,963312,7,0,None,None,None,None,None
1931,980922,34.91,31.57,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963278,None,None,963314,7,1,None,None,None,None,None
1932,980924,110.65,154.24,2019-01-27 10:00:00,2019-01-27 11:00:00,1,963280,None,None,963316,27,8,None,None,None,None,None
1933,980926,31.72,16.59,2019-01-26 17:00:00,2019-01-26 18:00:00,1,963282,None,None,963318,8,0,None,None,None,None,None
1934,980928,34.31,55.69,2019-01-26 18:00:00,2019-01-26 19:00:00,1,963284,None,None,963320,18,2,None,None,None,None,None
1935,980930,95.58,127.17,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963286,None,None,963322,37,8,None,None,None,None,None
1936,980932,178.63,29.85,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963288,None,None,963324,9,0,None,None,None,None,None


In [7]:
stores = pd.read_csv("../data/2019-01-31 s24/store.csv")
stores = stores.drop(["created_at", "updated_at", "whitelabel_id"], axis=1)
stores = stores.rename(columns=lambda name: "store_" + name)
stores.tail(10)

/home/gionata/github/analitico/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,store_id,store_name,store_province,store_lat,store_lng,store_area,store_ref_id
479943,963276,esselunga,MI,45.486030,9.164167,MI7,3209
479944,963278,conad superstore,RN,44.049117,12.566428,RN1,5215
479945,963280,eurospin,RM,41.895862,12.569003,RM6,5030
479946,963282,pam,TS,45.638624,13.768415,TS1,2048
479947,963284,carrefour,TO,45.066167,7.630242,TO2,1214
479948,963286,carrefour market,RM,41.861126,12.442734,RM4,330
479949,963288,esselunga,RM,41.895347,12.574626,RM6,5050
479950,963290,lidl,RM,41.915730,12.406670,RM11,5265
479951,963292,lidl,MI,45.499316,9.188497,MI3,5228
479952,963294,esselunga,MI,45.483443,9.189393,MI1,4447


In [8]:
orders = pd.merge(orders, stores, on="store_id")
orders.tail(10)

,order_id,order_amount,order_volume,order_deliver_at_start,order_deliver_at_end,order_fulfillment_type,store_id,courier_id,picker_id,customer_id,items_total,items_with_variable_weight,order_opened_at,order_first_detail_picked_at,order_last_detail_picked_at,order_first_try_pay_at,order_delivered_at,store_name,store_province,store_lat,store_lng,store_area,store_ref_id
1927,980914,34.99,115.40,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963270,None,None,963306,21,1,None,None,None,None,None,eurospin,RM,41.895421,12.395106,RM11,5032
1928,980916,45.66,180.51,2019-01-27 09:00:00,2019-01-27 10:00:00,1,963272,None,None,963308,8,0,None,None,None,None,None,lidl,RM,41.831100,12.581730,RM9,5267
1929,980918,31.72,33.57,2019-01-27 12:00:00,2019-01-27 13:00:00,1,963274,None,None,963310,11,1,None,None,None,None,None,carrefour market,TO,45.063749,7.576171,TO4,1231
1930,980920,43.43,13.60,2019-01-27 15:00:00,2019-01-27 16:00:00,3,963276,64456,12,963312,7,0,None,None,None,None,None,esselunga,MI,45.486030,9.164167,MI7,3209
1931,980922,34.91,31.57,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963278,None,None,963314,7,1,None,None,None,None,None,conad superstore,RN,44.049117,12.566428,RN1,5215
1932,980924,110.65,154.24,2019-01-27 10:00:00,2019-01-27 11:00:00,1,963280,None,None,963316,27,8,None,None,None,None,None,eurospin,RM,41.895862,12.569003,RM6,5030
1933,980926,31.72,16.59,2019-01-26 17:00:00,2019-01-26 18:00:00,1,963282,None,None,963318,8,0,None,None,None,None,None,pam,TS,45.638624,13.768415,TS1,2048
1934,980928,34.31,55.69,2019-01-26 18:00:00,2019-01-26 19:00:00,1,963284,None,None,963320,18,2,None,None,None,None,None,carrefour,TO,45.066167,7.630242,TO2,1214
1935,980930,95.58,127.17,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963286,None,None,963322,37,8,None,None,None,None,None,carrefour market,RM,41.861126,12.442734,RM4,330
1936,980932,178.63,29.85,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963288,None,None,963324,9,0,None,None,None,None,None,esselunga,RM,41.895347,12.574626,RM6,5050


In [9]:
customers = pd.read_csv("../data/2019-01-31 s24/customer.csv")
customers = customers.drop(["whitelabel_id"], axis=1)
customers = customers.rename(columns=lambda name: "customer_" + name)
customers.tail(5)

,customer_id,customer_province,customer_lat,customer_lng,customer_area,customer_ztl,customer_ref_id,customer_has_subscription,customer_created_at,customer_updated_at
479948,963322,RM,41.864045,12.461806,RM4,0,1129307,0,2019-01-26 14:05:02,2019-01-26 14:05:02
479949,963324,RM,41.873987,12.600522,RM6,0,1395300,0,2019-01-26 14:08:53,2019-01-26 14:08:53
479950,963326,RM,41.916467,12.411026,RM11,0,1331711,0,2019-01-26 14:11:47,2019-01-26 14:11:47
479951,963328,MI,45.491486,9.218570,MI3,0,380992,0,2019-01-26 14:12:27,2019-01-26 14:12:27
479952,963330,MI,45.461251,9.189844,MI1,20,185246,0,2019-01-26 14:13:12,2019-01-26 14:13:12


In [10]:
orders = pd.merge(orders, customers, on="customer_id")
orders.tail(10)

,order_id,order_amount,order_volume,order_deliver_at_start,order_deliver_at_end,order_fulfillment_type,store_id,courier_id,picker_id,customer_id,items_total,items_with_variable_weight,order_opened_at,order_first_detail_picked_at,order_last_detail_picked_at,order_first_try_pay_at,order_delivered_at,store_name,store_province,store_lat,store_lng,store_area,store_ref_id,customer_province,customer_lat,customer_lng,customer_area,customer_ztl,customer_ref_id,customer_has_subscription,customer_created_at,customer_updated_at
1927,980914,34.99,115.40,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963270,None,None,963306,21,1,None,None,None,None,None,eurospin,RM,41.895421,12.395106,RM11,5032,RM,41.915274,12.411252,RM11,0,915159,0,2019-01-26 13:58:50,2019-01-26 13:58:50
1928,980916,45.66,180.51,2019-01-27 09:00:00,2019-01-27 10:00:00,1,963272,None,None,963308,8,0,None,None,None,None,None,lidl,RM,41.831100,12.581730,RM9,5267,RM,41.852669,12.586698,RM9,0,1434568,0,2019-01-26 13:59:16,2019-01-26 13:59:16
1929,980918,31.72,33.57,2019-01-27 12:00:00,2019-01-27 13:00:00,1,963274,None,None,963310,11,1,None,None,None,None,None,carrefour market,TO,45.063749,7.576171,TO4,1231,TO,45.065400,7.539286,TO4,0,1374832,0,2019-01-26 13:59:23,2019-01-26 13:59:23
1930,980920,43.43,13.60,2019-01-27 15:00:00,2019-01-27 16:00:00,3,963276,64456,12,963312,7,0,None,None,None,None,None,esselunga,MI,45.486030,9.164167,MI7,3209,MI,45.461949,9.149044,MI7,0,1264403,0,2019-01-26 14:01:08,2019-01-26 14:01:08
1931,980922,34.91,31.57,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963278,None,None,963314,7,1,None,None,None,None,None,conad superstore,RN,44.049117,12.566428,RN1,5215,RN,44.044881,12.562022,RN1,0,1254117,0,2019-01-26 14:02:01,2019-01-26 14:02:01
1932,980924,110.65,154.24,2019-01-27 10:00:00,2019-01-27 11:00:00,1,963280,None,None,963316,27,8,None,None,None,None,None,eurospin,RM,41.895862,12.569003,RM6,5030,RM,41.873987,12.600522,RM6,0,1395300,0,2019-01-26 14:02:56,2019-01-26 14:02:56
1933,980926,31.72,16.59,2019-01-26 17:00:00,2019-01-26 18:00:00,1,963282,None,None,963318,8,0,None,None,None,None,None,pam,TS,45.638624,13.768415,TS1,2048,TS,45.640750,13.795828,TS1,0,1261399,0,2019-01-26 14:04:23,2019-01-26 14:04:23
1934,980928,34.31,55.69,2019-01-26 18:00:00,2019-01-26 19:00:00,1,963284,None,None,963320,18,2,None,None,None,None,None,carrefour,TO,45.066167,7.630242,TO2,1214,TO,45.077672,7.626298,TO2,0,1434562,0,2019-01-26 14:04:36,2019-01-26 14:04:36
1935,980930,95.58,127.17,2019-01-26 20:00:00,2019-01-26 21:00:00,1,963286,None,None,963322,37,8,None,None,None,None,None,carrefour market,RM,41.861126,12.442734,RM4,330,RM,41.864045,12.461806,RM4,0,1129307,0,2019-01-26 14:05:02,2019-01-26 14:05:02
1936,980932,178.63,29.85,2019-01-27 11:00:00,2019-01-27 12:00:00,1,963288,None,None,963324,9,0,None,None,None,None,None,esselunga,RM,41.895347,12.574626,RM6,5050,RM,41.873987,12.600522,RM6,0,1395300,0,2019-01-26 14:08:53,2019-01-26 14:08:53


In [11]:
order.to_csv("orders-joined.csv")


NameError: name 'order' is not defined